In [1]:
# Pre-requisite 코드
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# 최적화 단계

loss_fn = nn.CrossEntropyLoss() # 손실함수 초기화
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # 옵티마이저 초기화


In [ ]:
# 학습 단계
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # 모델을 학습(train) 모드로 설정합니다 - 배치 정규화(Batch Normalization) 및 드롭아웃(Dropout) 레이어들에 중요합니다.
    # 이 예시에서는 없어도 되지만, 모범 사례를 위해 추가해두었습니다.
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 모델을 평가(eval) 모드로 설정합니다 - 배치 정규화(Batch Normalization) 및 드롭아웃(Dropout) 레이어들에 중요합니다.
    # 이 예시에서는 없어도 되지만, 모범 사례를 위해 추가해두었습니다.
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # torch.no_grad()를 사용하여 테스트 시 변화도(gradient)를 계산하지 않도록 합니다.
    # 이는 requires_grad=True로 설정된 텐서들의 불필요한 변화도 연산 및 메모리 사용량 또한 줄여줍니다.
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
# 최적화 단계
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.313075  [   64/60000]
loss: 2.296030  [ 6464/60000]
loss: 2.286244  [12864/60000]
loss: 2.270681  [19264/60000]
loss: 2.249615  [25664/60000]
loss: 2.227428  [32064/60000]
loss: 2.225612  [38464/60000]
loss: 2.199590  [44864/60000]
loss: 2.195327  [51264/60000]
loss: 2.145094  [57664/60000]
Test Error: 
 Accuracy: 45.2%, Avg loss: 2.155641 

Epoch 2
-------------------------------
loss: 2.172256  [   64/60000]
loss: 2.156169  [ 6464/60000]
loss: 2.106257  [12864/60000]
loss: 2.111447  [19264/60000]
loss: 2.069539  [25664/60000]
loss: 2.010281  [32064/60000]
loss: 2.026657  [38464/60000]
loss: 1.961990  [44864/60000]
loss: 1.960628  [51264/60000]
loss: 1.865983  [57664/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.885171 

Epoch 3
-------------------------------
loss: 1.923634  [   64/60000]
loss: 1.883555  [ 6464/60000]
loss: 1.781436  [12864/60000]
loss: 1.806005  [19264/60000]
loss: 1.702947  [25664/60000]
loss: 1.659935  [32064/600